In [7]:
import os
import glob
import xarray as xr
import pandas as pd
import numpy as np
from rasterio.mask import mask
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt
import rioxarray
import rasterio as rasterio
from pyproj import CRS

In [8]:
# Read the data directory
# cm_dir = r"D:\GOES_data_CM\2022"

# CHANGE YEAR HERE

cm_dir = r"D:\GOES_data_CM\2023"

In [9]:
# Find all NetCDF files in the directory
day_paths = glob.glob(os.path.join(cm_dir, '*'),recursive=True)
# day_paths

In [10]:
# lat_lon_nc = xr.open_dataset(r"D:\goes16_abi_conus_lat_lon\goes16_abi_conus_lat_lon.nc")
# transposed_data = lat_lon_nc.transpose('columns', 'rows')
# goes_16_lat = transposed_data['latitude']
# goes_16_lon = transposed_data['longitude']
# goes_16_lat

In [11]:
def nan_to_mask(array):
    """
    Convert NaN values in an array to a mask.

    Parameters:
    array (ndarray): Input array with NaN values.

    Returns:
    ndarray: Mask array where NaN values are replaced with 0 and non-NaN values with 1.
    """
    mask = np.isnan(array)
    return (~mask).astype(int)

In [12]:
# Keep track of yearly total coverage
yearly_total_coverage = None

# For each day within goes2go download file structure
for day in day_paths:
    # Get all scans for every day
    day_nc_files_every_scan = glob.glob(day + '/*/*.nc')

    # Get the current day for final file name (example: '001' or '272') depending on day
    day_string = str(day_nc_files_every_scan[0]).split('\\')[3] 

    daily_max_scans = len(day_nc_files_every_scan)
    print('Max scans for ' + str(day_string) + ' is ' + str(daily_max_scans))

    # Initialize containers for summed data and coverage frequency
    summed_data = None
    coverage_frequency = None

    badlist =[]
    
    ## Main aggregation loop
    # For every scan in a day
    for scan in day_nc_files_every_scan:
        # Open the .nc file using xarray
        with xr.open_dataset(scan) as ds:
            # print(str(scan) + " contains nan values")
            # print(np.isnan(ds.BCM.values).any())
            if np.count_nonzero(np.isnan(ds.BCM.values)) > 47162: #np.isnan(ds.BCM.values).any(): # If the scan contains missing data
                # print("DEBUG num nans is = ")
                # print(str(np.count_nonzero(np.isnan(ds.BCM.values))))
                print("scan " + str(scan) + " was skipped due to incomplete scan")
                badlist = badlist.append(scan)
                # x = ds.BCM.values
                # continue
                if coverage_frequency is None:
                    coverage_frequency = nan_to_mask(ds.BCM.values)
                else:
                    # Get the locations of the good data
                    coverage_frequency += nan_to_mask(ds.BCM.values)

                # good_data_mask = nan_to_mask(ds.BCM.values)

                # # Add BCM values for the current .nc file to summed_data
                # if summed_data is None:
                #     # Initialize summed_data if it hasn't been initialized yet
                #     summed_data = ds.BCM.values * good_data_mask
                # else:
                #     # Add BCM values to summed_data
                #     summed_data += ds.BCM.values * good_data_mask           
            else: # If the scan is complete (i.e. has data for every pixel)
                if coverage_frequency is None:
                    # a += b
                    # np.add(a, b, out=a, casting="unsafe")
                    coverage_frequency = np.ones_like(ds.BCM.values)
                else:
                    np.add(coverage_frequency, np.ones_like(ds.BCM.values), out=coverage_frequency, casting="unsafe")
                    # coverage_frequency += np.ones_like(ds.BCM.values)

                if summed_data is None:
                    # Initialize summed_data if it hasn't been initialized yet
                    summed_data = ds.BCM.values
                else:
                    # Add BCM values to summed_data
                    summed_data += ds.BCM.values
        
    # Calculate the true cloud frequency accounting for missing data
    final_cloud_frequency = summed_data / coverage_frequency

    # Obtain the satellite height from the last scan in the day
    sat_height = ds.goes_imager_projection.attrs["perspective_point_height"]
    # Create a pyproj CRS
    cc = CRS.from_cf(ds.goes_imager_projection.attrs)

    # changed from summed_data
    # summed_dataset = xr.Dataset({'DCF': (('y', 'x'), final_cloud_frequency)},
    #                         coords={'x': ds.x.values * sat_height, 'y': ds.y.values * sat_height})
    summed_dataset = xr.Dataset({'Cloud_Frequency': (('y', 'x'), summed_data)},
                            coords={'x': ds.x.values * sat_height, 'y': ds.y.values * sat_height})

    summed_dataset.rio.write_crs(cc.to_string(), inplace=True)

    day_final = summed_dataset.rio.reproject("EPSG:3857") #9822

    # Save the aggregated data to a new NetCDF file
    day_final.to_netcdf('D:/GOES_CF_NCs/2022' + day_string + '.nc') # Summed_dataset


    # Calculate yearly coverage data
    if yearly_total_coverage is None:
        yearly_total_coverage = coverage_frequency
    else:
        yearly_total_coverage += coverage_frequency

    # Time to run for a full year: 215 Minutes (~4 hours)

Max scans for 001 is 288
Max scans for 002 is 288
Max scans for 003 is 288
Max scans for 004 is 288
Max scans for 005 is 288
Max scans for 006 is 288
Max scans for 007 is 288
Max scans for 008 is 288
Max scans for 009 is 288
Max scans for 010 is 288
Max scans for 011 is 288
Max scans for 012 is 288
Max scans for 013 is 288
Max scans for 014 is 288
Max scans for 015 is 288
Max scans for 016 is 288
Max scans for 017 is 288
Max scans for 018 is 288
Max scans for 019 is 288
Max scans for 020 is 288
Max scans for 021 is 288
Max scans for 022 is 288
Max scans for 023 is 288
Max scans for 024 is 288
Max scans for 025 is 288
Max scans for 026 is 288
Max scans for 027 is 288
Max scans for 028 is 288
Max scans for 029 is 288
Max scans for 030 is 288
Max scans for 031 is 288
Max scans for 032 is 288
Max scans for 033 is 288
Max scans for 034 is 288
Max scans for 035 is 287
scan D:\GOES_data_CM\2023\035\11\OR_ABI-L2-ACMC-M4_G16_s20230351115223_e20230351115223_c20230351122339.nc was skipped due to 

In [23]:
# print(str(np.count_nonzero(np.isnan(ds.BCM.values))))
# len(coverage_frequency)

1500

In [14]:
# Make yearly coverage nc
YTC = xr.Dataset({'Yearly_Total_Coverage': (('y', 'x'), yearly_total_coverage)},
                        coords={'x': ds.x.values * sat_height, 'y': ds.y.values * sat_height})

YTC.rio.write_crs(cc.to_string(), inplace=True)

year_coverage_final = YTC.rio.reproject("EPSG:3857")

# Save the aggregated data to a new NetCDF file
year_coverage_final.to_netcdf('D:/GOES_YEARLY_COVERAGE/2023.nc') # aggregated coverage data

In [15]:
nc_dir = 'D:/GOES_CF_NCs/2023'

In [16]:
# Read all the nc file paths
nc_paths = glob.glob(os.path.join(nc_dir, '*.nc'))

# Count the .nc files and store as a variable

def count_files_with_extension(folder_path, extension):
    # Get the list of files in the specified folder
    files = os.listdir(folder_path)

    # Filter files with the specified extension
    nc_files = [file for file in files if file.endswith(extension)]

    # Count the number of filtered files
    count = len(nc_files)

    return count

# Define the extension type

extension = '.nc'

# Call the function to count files with the specified extension
file_count = count_files_with_extension(nc_dir, extension)

print(f'The number of files with {extension} extension in {nc_dir} is: {file_count}')

# Stacking the images
img_stack = np.zeros(shape=(file_count, 1408, 2553))

for i, nc_path in enumerate(nc_paths):
    src = rasterio.open(nc_path)
    img = src.read()
    img_stack[i, :, :] = img
    src.close()
    
    
#open the bil file
src = rasterio.open(nc_paths[0])

#Read metadata
profile = src.profile
profile

#Make a composite raster of the temp data
profile.update(
    driver='GTiff',
    count=file_count)

out_path = r'D:/GOES_CF_RASTERS/2023_Cloud_Frequency.tif'

with rasterio.open(out_path, 'w', **profile) as dst:
    dst.write(img_stack)
    
src.close()

# Time to run ~17 minutes

The number of files with .nc extension in D:/GOES_CF_NCs/2023 is: 365
